This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests
import pprint

In [ ]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

quandl_dataset_url="https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json"

start_date='2017-01-01'
end_date='2017-01-03'
payload = {'start_date': f'{start_date}', 'end_date': f'{end_date}', 'order':'asc'}
resp = requests.get(quandl_dataset_url, params=payload)
resp_json=resp.json()
#print(resp_json)
#pprint.pprint(resp_json)

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

#print(type(resp_json))
#<class 'dict'>



In [ ]:
print (API_KEY)

NameError: name 'API_KEY' is not defined

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:

import requests

API_KEY = ''


# Get data from quandl api
# pass start date , end date as parameters
# pass 'order':'asc' as parameter to make sure data is sorted in ascending order of data
# having data sorted by enable will enable calculating change between any two days
def get_data_from_api(url, start_date, end_date):
    payload = {'start_date': f'{start_date}', 'end_date': f'{end_date}', 'order': 'asc'}
    response = requests.get(url, params=payload)
    # use response.json to store in to a dictionary
    response_json = response.json()
    return response_json


# Find index of each data column based on the column name
def index_finder(names_list, column_name):
    return names_list.index(column_name)


# Find median of a given list
# Sort the list
# If number of elements is even find average of the two middle elements
# if number of elements is odd, find the middle element
def find_median(input_list):
    sorted_list = sorted(input_list)
    list_len = len(input_list)
    if list_len % 2 == 0:
        mid1 = sorted_list[list_len//2]
        mid2 = sorted_list[list_len//2-1]
        med = (mid1 + mid2)/2
    else:
        med = sorted_list[list_len//2]
    return med


# Traverse the data list to calculate needed metrics
def traverse_data_to_calculate_metrics(all_data, column_names_list):

    # The data in list is arranged in the same order as the column name
    # find index for each of the needed data element like Open, High etc
    column_index_for_open = index_finder(column_names_list, 'Open')
    column_index_for_high = index_finder(column_names_list, 'High')
    column_index_for_low = index_finder(column_names_list, 'Low')
    column_index_for_close = index_finder(column_names_list, 'Close')
    column_index_for_traded_volume = index_finder(column_names_list, 'Traded Volume')

    # Find length of array, used to traverse the list and used in calculating average
    number_of_days = len(all_data)

    # get all data values for the first date in the list
    first_day_opening_price = all_data[0][column_index_for_open]
    first_day_opening_high = all_data[0][column_index_for_high]
    first_day_opening_low = all_data[0][column_index_for_low]
    first_day_change = round(first_day_opening_high - first_day_opening_low, 2)
    first_day_close = all_data[0][column_index_for_close]
    first_day_trading_volume = all_data[0][column_index_for_traded_volume]

    # Initialize all metrics like highest_opening_price_so_far, lowest_opening_price_so_far to value from first date
    highest_opening_price_so_far = first_day_opening_price
    lowest_opening_price_so_far = first_day_opening_price
    largest_change_so_far = first_day_change
    prev_day_close = first_day_close
    largest_two_days_close_diff = round(first_day_close - prev_day_close, 2)
    sum_traded_volume = first_day_trading_volume

    # start iteration from second date
    for i in range(1, number_of_days):
        # temp variable to store current day data
        d = all_data[i]

        # take care of missing values in 'open'
        # if data is missing for 'open', use previous day close
        if d[column_index_for_open]:
            curr_day_opening_price = d[column_index_for_open]
        else:
            curr_day_opening_price = prev_day_close

        # find maximum of current day opening price and highest_opening_price_so_far
        if curr_day_opening_price > highest_opening_price_so_far:
            highest_opening_price_so_far = curr_day_opening_price

        # find minimum of current day opening price and lowest_opening_price_so_far
        if curr_day_opening_price < lowest_opening_price_so_far:
            lowest_opening_price_so_far = curr_day_opening_price

        # find difference between high and low for the current day
        curr_day_high = d[column_index_for_high]
        curr_day_low = d[column_index_for_low]
        curr_day_change = round(curr_day_high - curr_day_low, 2)

        # find maximum of current day change and largest_change_so_far
        if curr_day_change > largest_change_so_far:
            largest_change_so_far = curr_day_change

        # find difference between current day close and previous day close
        curr_day_close = d[column_index_for_close]
        curr_close_diff = round(abs(curr_day_close - prev_day_close), 2)

        # find maximum of current day close and largest_two_days_close_diff
        if curr_close_diff > largest_two_days_close_diff:
            largest_two_days_close_diff = curr_close_diff
        prev_day_close = curr_day_close

        # add trading volume
        curr_day_trading_volume = d[column_index_for_traded_volume]
        sum_traded_volume += curr_day_trading_volume

    print ('Calculated Metrics')

    # Calculate what the highest and lowest opening prices were for the stock in this period
    print ('highest_opening_price_so_far', highest_opening_price_so_far)
    print ('lowest_opening_price_so_far', lowest_opening_price_so_far)

    # What was the largest change in any one day (based on High and Low price)?
    print ('largest_change_so_far', largest_change_so_far)

    # What was the largest change between any two days (based on Closing Price)?
    print ('final largest_two_days_close_diff', largest_two_days_close_diff)

    # find average trading volume
    average_traded_volume = round(sum_traded_volume / number_of_days)
    # What was the average daily trading volume during this year?
    print ('average_traded_volume', average_traded_volume)

    # median - step 1: traverse the list and store the traded volume to a new list
    traded_volume_list = [i[column_index_for_traded_volume] for i in all_data]
    # median - step 2: call find median
    median_traded_volume = find_median(traded_volume_list)
    # What was the median trading volume during this year
    print ('median_traded_volume', round(median_traded_volume))


def main():
    # Initialize url for ticker AFX_X
    quandl_dataset_url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json"
    start_date = '2017-01-01'
    end_date = '2017-12-31'

    # collect data from api for year 2017
    quandset_api_response = get_data_from_api(quandl_dataset_url, start_date, end_date)
    # get data values and column names
    all_data = quandset_api_response['dataset']['data']
    column_names_list = quandset_api_response['dataset']['column_names']
    # iterate through data and derive metrics
    traverse_data_to_calculate_metrics(all_data, column_names_list)


main()


Calculated Metrics
highest_opening_price_so_far 53.11
lowest_opening_price_so_far 34.0
largest_change_so_far 2.81
final largest_two_days_close_diff 2.56
average_traded_volume 89124
median_traded_volume 76286
